In [2]:
import pandas as pd
import numpy as np
np.random.seed(0)
import seaborn as sns
import matplotlib.pyplot as plt
%matplotlib inline
from sklearn.model_selection import train_test_split, cross_val_score
from sklearn.ensemble import AdaBoostClassifier, GradientBoostingClassifier
from sklearn.metrics import accuracy_score, f1_score, confusion_matrix, classification_report

## time for PCA

In [3]:
df = pd.read_csv('5k_clean.csv')

In [4]:
df.columns

Index(['Unnamed: 0', 'state', 'agency', 'case_number', 'UCR_code',
       'attempt_or_complete', 'DAC_1', 'location_type', 'weapon_type_1',
       'bias_1', 'perp_seq_num', 'perp_age', 'perp_sex', 'perp_race',
       'perp_ethn', 'num_ori_matches_x', 'victim_seq_num', 'ucr_1', 'vic_type',
       'vic_age', 'vic_sex', 'vic_race', 'vic_ethn', 'num_ori_matches_y'],
      dtype='object')

In [5]:
df.head()

,Unnamed: 0,state,agency,case_number,UCR_code,attempt_or_complete,DAC_1,location_type,weapon_type_1,bias_1,...,perp_ethn,num_ori_matches_x,victim_seq_num,ucr_1,vic_type,vic_age,vic_sex,vic_race,vic_ethn,num_ori_matches_y
0,3261531,24,MOKPD0000,830BABEDG2 N,Assault,C,N,14,20,88,...,N,2.0,1,13A,I,37.0,M,W,U,1.0
1,6262040,41,TNMPD0000,BJ0K39W9HPMF,Assault,C,N,20,40.0,88,...,N,1.0,1,13B,I,21,F,B,N,1.0
2,6207246,41,TNMPD0000,1X1M0UFO66HH,Assault,C,A,20,95.0,88,...,N,1.0,1,13B,I,57,F,B,N,1.0
3,5617093,41,TN0190100,2W1I027LUX6A,Theft,C,N,14,99.0,88,...,N,2.0,1,120,I,26,M,W,N,1.0
4,6260994,41,TNMPD0000,BJ0K39U398MF,Assault,C,N,20,40.0,88,...,N,1.0,1,13B,I,63,M,W,N,1.0


- get:

    - df with X dummies and Y var, but not Y dummies
        - calling dummies() with Y included destroys the Y var, anyway
    - list of X vars with dummies


## interesting note: PCA params
PCA was hitting some errors with solver shape sizes defaulting to 5000, and i have ~6000 columns.

turns out it was setting 5000 as min_components because i had 5000 rows. I could change the solver, certainly, but now would be a good time to bring down a larger sample from the full set.

In [22]:
df = pd.read_csv('merged.csv')
len(df)

8505901

Index(['agency', 'case_number', 'UCR_code', 'attempt_or_complete', 'DAC_1',
       'location_type', 'weapon_type_1', 'bias_1', 'perp_seq_num', 'perp_age',
       'perp_sex', 'perp_race', 'perp_ethn', 'num_ori_matches_x', 'state',
       'victim_seq_num', 'ucr_1', 'vic_type', 'vic_age', 'vic_sex', 'vic_race',
       'vic_ethn', 'num_ori_matches_y'],
      dtype='object')

In [39]:
df.isna().sum()

agency                       0
case_number                  0
UCR_code                     0
attempt_or_complete          0
DAC_1                        0
location_type                0
weapon_type_1          6562938
bias_1                       0
perp_seq_num                 0
perp_age               2070146
perp_sex               2070146
perp_race              2070146
perp_ethn              6135982
num_ori_matches_x      1409760
state                        0
victim_seq_num               0
ucr_1                        0
vic_type                     0
vic_age                2809639
vic_sex                2809639
vic_race               2809639
vic_ethn               3284705
num_ori_matches_y      1485683
dtype: int64

In [40]:
storage = df

In [77]:
len(df)

20000

In [61]:
df = storage

In [36]:
len(storage)

8505901

In [62]:
len(df)

8505901

In [63]:
len(df.dropna(axis=0))

364778

In [65]:
len(df)

364778

In [66]:
samp = df.sample(20000,random_state=1)

In [72]:
df = samp

## below: consolidated into one cell for next notebook

In [ ]:
df = pd.read_csv('merged.csv')
df = df.drop(['Unnamed: 0','state_y','state_x'], axis=1) # drop unneeded cols
# df = df.drop(to_drop,axis=1)

# drop nulls?
df = df.dropna()


# df with dummies

# remove Y to avoid Y dummies
target = df['UCR_code']
df = df.drop('UCR_code',axis=1)

# dummify
df = pd.get_dummies(df,drop_first=True)

features = list(df.columns) # get dummy list

df['UCR_code'] = target # this should replace the original undummied Y

/Users/mark/opt/anaconda3/lib/python3.7/site-packages/IPython/core/interactiveshell.py:3058: DtypeWarning: Columns (8,15,21,24) have mixed types. Specify dtype option on import or set low_memory=False.
  interactivity=interactivity, compiler=compiler, result=result)


In [74]:
features = list(df.columns) # get dummy list

In [75]:
len(df.columns)

21365

In [54]:
df['UCR_code'] = target # this should replace the original undummied Y

In [55]:
# features = ['cylinders', 'displacement', 'horsepower', 'weight',
#        'acceleration']
# preprocess the data 
from sklearn.preprocessing import StandardScaler
# Separating out the features
x = df.loc[:, features].values
# Separating out the target
y = df.loc[:,['UCR_code']].values

# what if we just change it 

In [56]:
x = StandardScaler().fit_transform(x)

In [76]:
# create principal components 
from sklearn.decomposition import PCA
pca = PCA()
principalComponents = pca.fit_transform(x)
principalDf = pd.DataFrame(data = principalComponents, columns = features)

ValueError: Shape of passed values is (10000, 10000), indices imply (10000, 21365)

In [ ]:
pca.explained_variance_ratio_

In [ ]:
pca.explained_variance_

In [ ]:
principalDf.iloc[:,:3].head()

In [ ]:
eig_values = pca.explained_variance_
eig_vectors = pca.components_
print(eig_values)
print(eig_vectors)

In [ ]:
# examine the first pricipal component
eig_vectors[0]

In [ ]:
# examine the components
pc1 = pca.components_[0]
pc2 = pca.components_[1]
# the .components attribute shows principal axes in feature space, representing the directions of maximum variance in the data. 
#The components are sorted by explained_variance_


In [ ]:
print(pc1)
print(pc2)

In [ ]:
# get the loadings
structure_loading_1 = pc1* np.sqrt(eig_values[0])
str_loading_1 = pd.Series(structure_loading_1, index=features)
str_loading_1

In [ ]:
str_loading_1.sort_values(ascending=False)

In [ ]:
index = np.arange(6)
plt.bar(index, pca.explained_variance_ratio_)
plt.title('Scree plot for PCA')
plt.xlabel('Num of components')
plt.ylabel('proportion of explained variance')

In [ ]:
# plotting screeplots 
#print(pca.explained_variance_ratio_)
#print(pca.explained_variance_)

plt.plot(np.cumsum(pca.explained_variance_ratio_))
plt.xlabel('number of components')
plt.ylabel('explained variance ratio')

In [ ]:
to_drop